In [ ]:
# ==============================
# 1. IMPORT LIBRARIES
# ==============================
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing import sequence

# ==============================
# 2. PARAMETERS (HIGH LEVEL)
# ==============================
max_features = 15000    # Vocabulary size
maxlen = 200            # Sequence length
embedding_dim = 64      # Embedding dimension
lstm_units = 64         # LSTM units
cnn_filters = 64        # CNN filters

# ==============================
# 3. LOAD AND PREPROCESS DATA
# ==============================
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

# ==============================
# 4. BUILD THE EXPERT MODEL
# ==============================
model = Sequential()

# Embedding
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Dropout(0.3))

# Bidirectional LSTM
model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
model.add(BatchNormalization())

# 1D CNN Layer
model.add(Conv1D(filters=cnn_filters, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())

# Dense layers with Dropout
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

# ==============================
# 5. COMPILE THE MODEL
# ==============================
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')]
)

# ==============================
# 6. CALLBACKS
# ==============================
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6)
]

# ==============================
# 7. TRAIN THE MODEL
# ==============================
history = model.fit(
    x_train, y_train,
    epochs=12,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks
)

# ==============================
# 8. EVALUATE MODEL
# ==============================
test_loss, test_acc, test_prec, test_rec, test_auc = model.evaluate(x_test, y_test, verbose=0)
print("\n===== TEST RESULTS =====")
print(f"Loss:      {test_loss:.4f}")
print(f"Accuracy:  {test_acc:.4f}")
print(f"Precision: {test_prec:.4f}")
print(f"Recall:    {test_rec:.4f}")
print(f"AUC:       {test_auc:.4f}")

# ==============================
# 9. PLOT TRAINING HISTORY
# ==============================
# Accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Precision
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.title('Model Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.show()

# Recall
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.title('Model Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.show()

# ==============================
# 10. PREDICTION ON SAMPLE REVIEW
# ==============================
sample_review = x_test[0]
sample_review_input = np.expand_dims(sample_review, axis=0)
prediction = model.predict(sample_review_input)[0][0]

print(f"\nPredicted Probability: {prediction:.4f}")
if prediction > 0.5:
    print("Predicted Sentiment: Positive")
else:
    print("Predicted Sentiment: Negative")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/12


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 113s 676ms/step - accuracy: 0.6408 - auc: 0.7009 - loss: 0.6073 - precision: 0.6406 - recall: 0.6337 - val_accuracy: 0.5926 - val_auc: 0.9456 - val_loss: 0.6288 - val_precision: 0.5481 - val_recall: 0.9972 - learning_rate: 0.0010
Epoch 2/12
157/157 ━━━━━━━━━━━━━━━━━━━━ 108s 687ms/step - accuracy: 0.9026 - auc: 0.9657 - loss: 0.2378 - precision: 0.9039 - recall: 0.9017 - val_accuracy: 0.8354 - val_auc: 0.9525 - val_loss: 0.5215 - val_precision: 0.7636 - val_recall: 0.9656 - learning_rate: 0.0010
Epoch 3/12
 64/157 ━━━━━━━━━━━━━━━━━━━━ 59s 645ms/step - accuracy: 0.9442 - auc: 0.9832 - loss: 0.1697 - precision: 0.9476 - recall: 0.9408 